In [1]:
import sys
import cv2
import numpy as np
import json
import os
from datetime import datetime
from typing import List, Tuple, Dict, Optional

from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *

import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure


class LeafDiseaseAnalyzer:
    def __init__(self, pixels_per_cm: float = 62.0, min_leaf_area_cm: float = 0.5):
        self.pixels_per_cm = pixels_per_cm
        self.min_leaf_area_cm = min_leaf_area_cm

        self.color_ranges = {
            'healthy_green': {'lower': [35, 40, 40], 'upper': [85, 255, 255]},
            'light_green': {'lower': [25, 30, 50], 'upper': [95, 255, 255]},
            'yellow_damage': {'lower': [15, 50, 50], 'upper': [35, 255, 255]},
            'brown_damage': {'lower': [0, 50, 50], 'upper': [20, 255, 150]},
            'white_damage': {'lower': [0, 0, 200], 'upper': [180, 50, 255]}
        }

        self.results = {
            'total_leaves': 0,
            'leaves': [],
            'overall_disease_percentage': 0.0,
            'pixels_per_cm': pixels_per_cm,
            'filtered_out_leaves': 0,
            'min_leaf_area_cm': min_leaf_area_cm
        }

    def preprocess_image(self, image_path: str) -> Tuple[np.ndarray, np.ndarray]:
        img = cv2.imread(image_path)
        if img is None:
            raise FileNotFoundError(f"Не удалось загрузить изображение: {image_path}")
        
        height, width = img.shape[:2]
        if width > 1200:
            scale = 1200 / width
            new_width = int(width * scale)
            new_height = int(height * scale)
            img = cv2.resize(img, (new_width, new_height))

        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        limg = cv2.merge((cl, a, b))
        img_processed = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        img_processed = cv2.GaussianBlur(img_processed, (3, 3), 0)
        hsv = cv2.cvtColor(img_processed, cv2.COLOR_BGR2HSV)
        return img, hsv

    def create_color_mask(self, hsv: np.ndarray, color_name: str) -> np.ndarray:
        lower = np.array(self.color_ranges[color_name]['lower'])
        upper = np.array(self.color_ranges[color_name]['upper'])
        mask = cv2.inRange(hsv, lower, upper)
        kernel = np.ones((3, 3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        return mask

    def detect_individual_leaves(self, green_mask: np.ndarray) -> List[Tuple[np.ndarray, float, float]]:
        kernel = np.ones((5, 5), np.uint8)
        cleaned_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)
        cleaned_mask = cv2.morphologyEx(cleaned_mask, cv2.MORPH_CLOSE, kernel)
        
        contours, _ = cv2.findContours(cleaned_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        leaf_info = []
        filtered_count = 0

        for contour in contours:
            area_px = cv2.contourArea(contour)
            area_cm = area_px / (self.pixels_per_cm ** 2)

            if area_cm < self.min_leaf_area_cm:
                filtered_count += 1
                continue

            epsilon = 0.01 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            if len(approx) >= 3:
                leaf_info.append((approx, area_px, area_cm))
            else:
                filtered_count += 1

        leaf_info.sort(key=lambda x: x[1], reverse=True)
        return leaf_info

    def calculate_leaf_metrics(self, contour: np.ndarray,
                               contour_area_px: float,
                               contour_area_cm: float,
                               green_mask: np.ndarray,
                               damage_mask: np.ndarray) -> Optional[Dict]:
        leaf_mask = np.zeros_like(green_mask)
        cv2.drawContours(leaf_mask, [contour], -1, 255, -1)

        if cv2.countNonZero(leaf_mask) == 0:
            return None

        green_in_leaf = cv2.bitwise_and(green_mask, leaf_mask)
        green_area_px = cv2.countNonZero(green_in_leaf)
        green_area_cm = green_area_px / (self.pixels_per_cm ** 2)

        damage_in_leaf = cv2.bitwise_and(damage_mask, leaf_mask)
        damage_area_px = cv2.countNonZero(damage_in_leaf)
        damage_area_cm = damage_area_px / (self.pixels_per_cm ** 2)

        green_without_damage = cv2.bitwise_and(green_in_leaf, cv2.bitwise_not(damage_in_leaf))
        green_area_px_corrected = cv2.countNonZero(green_without_damage)
        green_area_cm_corrected = green_area_px_corrected / (self.pixels_per_cm ** 2)

        total_leaf_area_px = contour_area_px
        total_leaf_area_cm = contour_area_cm

        if total_leaf_area_px > 0:
            disease_percentage_total = (damage_area_px / total_leaf_area_px) * 100
        else:
            disease_percentage_total = 0

        return {
            'contour_area_px': total_leaf_area_px,
            'contour_area_cm': total_leaf_area_cm,
            'healthy_green_area_px': green_area_px_corrected,
            'healthy_green_area_cm': green_area_cm_corrected,
            'damage_area_px': damage_area_px,
            'damage_area_cm': damage_area_cm,
            'disease_percentage_by_damage': disease_percentage_total,
            'damage_mask': damage_in_leaf,
            'healthy_mask': green_without_damage,
            'leaf_mask': leaf_mask
        }

    def analyze_image(self, image_path: str, use_convex_hull: bool = False) -> Dict:
        try:
            img, hsv = self.preprocess_image(image_path)

            green_mask = self.create_color_mask(hsv, 'healthy_green')
            light_green_mask = self.create_color_mask(hsv, 'light_green')
            green_mask = cv2.bitwise_or(green_mask, light_green_mask)
            
            yellow_damage = self.create_color_mask(hsv, 'yellow_damage')
            brown_damage = self.create_color_mask(hsv, 'brown_damage')
            white_damage = self.create_color_mask(hsv, 'white_damage')

            damage_mask = cv2.bitwise_or(yellow_damage, brown_damage)
            damage_mask = cv2.bitwise_or(damage_mask, white_damage)
            green_mask = cv2.bitwise_and(green_mask, cv2.bitwise_not(damage_mask))

            leaf_info = self.detect_individual_leaves(green_mask)

            if not leaf_info:
                return self.results

            results_list = []
            all_damage_mask = np.zeros_like(damage_mask)

            for i, (contour, area_px, area_cm) in enumerate(leaf_info):
                if use_convex_hull:
                    contour = cv2.convexHull(contour)
                    area_px = cv2.contourArea(contour)
                    area_cm = area_px / (self.pixels_per_cm ** 2)

                metrics = self.calculate_leaf_metrics(
                    contour=contour,
                    contour_area_px=area_px,
                    contour_area_cm=area_cm,
                    green_mask=green_mask,
                    damage_mask=damage_mask
                )

                if metrics is None:
                    continue

                if metrics['contour_area_cm'] < self.min_leaf_area_cm:
                    continue

                results_list.append(metrics)
                all_damage_mask = cv2.bitwise_or(all_damage_mask, metrics['damage_mask'])

            visualization = self.create_visualization(
                img,
                [info[0] for info in leaf_info[:len(results_list)]],
                results_list,
                all_damage_mask
            )

            if results_list:
                total_damage_percentage = np.mean([r['disease_percentage_by_damage'] for r in results_list])
                total_healthy_area = sum([r['healthy_green_area_cm'] for r in results_list])
                total_damage_area = sum([r['damage_area_cm'] for r in results_list])
                total_contour_area = sum([r['contour_area_cm'] for r in results_list])
            else:
                total_damage_percentage = 0
                total_healthy_area = 0
                total_damage_area = 0
                total_contour_area = 0

            self.results = {
                'total_leaves': len(results_list),
                'leaves': results_list,
                'overall_disease_percentage_damage': total_damage_percentage,
                'total_healthy_area_cm': total_healthy_area,
                'total_damage_area_cm': total_damage_area,
                'total_contour_area_cm': total_contour_area,
                'pixels_per_cm': self.pixels_per_cm,
                'min_leaf_area_cm': self.min_leaf_area_cm,
                'filtered_out_leaves': len(leaf_info) - len(results_list),
                'visualization': visualization
            }

            return self.results
            
        except Exception as e:
            print(f"Ошибка при анализе: {e}")
            raise

    def create_visualization(self, img: np.ndarray, leaf_contours: List[np.ndarray],
                             results_list: List[Dict], damage_mask: np.ndarray) -> np.ndarray:
        vis_img = img.copy()
        damage_overlay = vis_img.copy()
        damage_overlay[damage_mask > 0] = (0, 0, 255)
        cv2.addWeighted(damage_overlay, 0.3, vis_img, 0.7, 0, vis_img)

        colors = [(0, 255, 0), (255, 255, 0), (255, 0, 255), (0, 255, 255), (255, 0, 0)]

        for i, (contour, result) in enumerate(zip(leaf_contours[:len(results_list)], results_list)):
            color = colors[i % len(colors)]
            cv2.drawContours(vis_img, [contour], -1, color, 2)
            
            M = cv2.moments(contour)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                cv2.putText(vis_img, str(i+1), (cx-10, cy-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        return vis_img


class HistoryItem:
    def __init__(self, filename, date, results, image_path):
        self.filename = filename
        self.date = date
        self.results = results
        self.image_path = image_path
        self.thumbnail = None
        
    def create_thumbnail(self, size=(80, 80)):
        try:
            pixmap = QPixmap(self.image_path)
            self.thumbnail = pixmap.scaled(*size, Qt.KeepAspectRatio, Qt.SmoothTransformation)
        except:
            pass


class LeafDiseaseAnalyzerGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.analyzer = None
        self.current_image_path = None
        self.results = None
        self.history_items = []
        self.initUI()
    
    def initUI(self):
        self.setWindowTitle('🍃 Анализатор листьев')
        self.setGeometry(100, 100, 1200, 700)
        
        # Центральный виджет
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        
        # Главный layout
        main_layout = QHBoxLayout()
        central_widget.setLayout(main_layout)
        
        # Левая панель - загрузка и анализ
        left_panel = QWidget()
        left_layout = QVBoxLayout()
        left_panel.setLayout(left_layout)
        left_panel.setMaximumWidth(400)
        
        # Заголовок
        title = QLabel('🍃 Анализатор листьев')
        title.setStyleSheet('''
            font-size: 20px;
            font-weight: bold;
            color: #27ae60;
            padding: 15px;
            background-color: #f0f8f0;
            border-radius: 8px;
            border-left: 5px solid #27ae60;
        ''')
        left_layout.addWidget(title)
        
        # Кнопки загрузки и анализа
        buttons_layout = QVBoxLayout()
        
        self.load_btn = QPushButton('📁 Загрузить изображение')
        self.load_btn.setStyleSheet(self.get_button_style('#3498db', '#2980b9'))
        self.load_btn.clicked.connect(self.load_image)
        buttons_layout.addWidget(self.load_btn)
        
        self.analyze_btn = QPushButton('🔬 Анализировать')
        self.analyze_btn.setStyleSheet(self.get_button_style('#27ae60', '#229954'))
        self.analyze_btn.clicked.connect(self.analyze)
        self.analyze_btn.setEnabled(False)
        buttons_layout.addWidget(self.analyze_btn)
        
        left_layout.addLayout(buttons_layout)
        
        # Информация о файле
        info_group = QGroupBox('📄 Информация')
        info_layout = QVBoxLayout()
        
        self.file_info = QLabel('Файл не загружен')
        self.file_info.setWordWrap(True)
        self.file_info.setStyleSheet('''
            padding: 10px;
            background-color: #f8f9fa;
            border-radius: 5px;
            border: 1px solid #dee2e6;
        ''')
        info_layout.addWidget(self.file_info)
        
        info_group.setLayout(info_layout)
        left_layout.addWidget(info_group)
        
        # Результаты анализа
        results_group = QGroupBox('📊 Результаты')
        results_layout = QVBoxLayout()
        
        self.results_text = QTextEdit()
        self.results_text.setReadOnly(True)
        self.results_text.setMaximumHeight(200)
        self.results_text.setStyleSheet('''
            font-family: monospace;
            font-size: 12px;
            background-color: #f8f9fa;
            border: 1px solid #dee2e6;
            border-radius: 5px;
            padding: 10px;
        ''')
        results_layout.addWidget(self.results_text)
        
        results_group.setLayout(results_layout)
        left_layout.addWidget(results_group)
        
        # Прогресс бар
        self.progress = QProgressBar()
        self.progress.hide()
        self.progress.setStyleSheet('''
            QProgressBar {
                border: 1px solid #dee2e6;
                border-radius: 3px;
                text-align: center;
            }
            QProgressBar::chunk {
                background-color: #27ae60;
                border-radius: 3px;
            }
        ''')
        left_layout.addWidget(self.progress)
        
        left_layout.addStretch()
        
        # Правая панель - изображения и история
        right_panel = QTabWidget()
        right_panel.setMinimumWidth(800)
        right_panel.setStyleSheet('''
            QTabWidget::pane {
                border: 1px solid #dee2e6;
                border-radius: 5px;
                padding: 10px;
                background-color: white;
            }
            QTabBar::tab {
                padding: 10px 20px;
                margin-right: 2px;
                border-top-left-radius: 5px;
                border-top-right-radius: 5px;
                font-weight: bold;
            }
            QTabBar::tab:selected {
                background-color: #27ae60;
                color: white;
            }
        ''')
        
        # Вкладка с изображениями
        images_tab = QWidget()
        images_layout = QHBoxLayout()
        
        # Оригинал
        original_widget = QWidget()
        original_layout = QVBoxLayout()
        original_layout.addWidget(QLabel('🖼 Оригинал'))
        self.original_label = QLabel()
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setStyleSheet('''
            border: 1px solid #dee2e6;
            background-color: #f8f9fa;
            border-radius: 5px;
            padding: 10px;
            min-height: 350px;
        ''')
        self.original_label.setText('Загрузите изображение')
        original_layout.addWidget(self.original_label)
        original_widget.setLayout(original_layout)
        images_layout.addWidget(original_widget)
        
        # Результат
        result_widget = QWidget()
        result_layout = QVBoxLayout()
        result_layout.addWidget(QLabel('📈 Результат'))
        self.result_label = QLabel()
        self.result_label.setAlignment(Qt.AlignCenter)
        self.result_label.setStyleSheet('''
            border: 1px solid #dee2e6;
            background-color: #f8f9fa;
            border-radius: 5px;
            padding: 10px;
            min-height: 350px;
        ''')
        self.result_label.setText('Анализ не выполнен')
        result_layout.addWidget(self.result_label)
        result_widget.setLayout(result_layout)
        images_layout.addWidget(result_widget)
        
        images_tab.setLayout(images_layout)
        right_panel.addTab(images_tab, '🖼 Изображения')
        
        # Вкладка с историей
        history_tab = QWidget()
        history_layout = QVBoxLayout()
        
        # Список истории
        self.history_list = QListWidget()
        self.history_list.setIconSize(QSize(60, 60))
        self.history_list.setSpacing(5)
        self.history_list.setStyleSheet('''
            QListWidget {
                border: 1px solid #dee2e6;
                border-radius: 5px;
                padding: 5px;
            }
            QListWidget::item {
                padding: 10px;
                border-bottom: 1px solid #f0f0f0;
            }
            QListWidget::item:hover {
                background-color: #f8f9fa;
            }
            QListWidget::item:selected {
                background-color: #e8f5e9;
            }
        ''')
        self.history_list.itemClicked.connect(self.load_from_history)
        history_layout.addWidget(self.history_list)
        
        # Кнопка очистки истории
        self.clear_btn = QPushButton('🗑 Очистить историю')
        self.clear_btn.setStyleSheet('''
            QPushButton {
                background-color: #e74c3c;
                color: white;
                padding: 10px;
                border-radius: 5px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #c0392b;
            }
        ''')
        self.clear_btn.clicked.connect(self.clear_history)
        history_layout.addWidget(self.clear_btn)
        
        history_tab.setLayout(history_layout)
        right_panel.addTab(history_tab, '📋 История')
        
        main_layout.addWidget(left_panel)
        main_layout.addWidget(right_panel)
        
        # Статус бар
        self.status_bar = QStatusBar()
        self.setStatusBar(self.status_bar)
        self.status_bar.showMessage('✅ Готов к работе')
    
    def get_button_style(self, color, hover):
        return f'''
            QPushButton {{
                background-color: {color};
                color: white;
                font-size: 14px;
                font-weight: bold;
                padding: 15px;
                border-radius: 8px;
                margin: 5px;
                border: none;
            }}
            QPushButton:hover {{
                background-color: {hover};
                border: 2px solid white;
            }}
            QPushButton:disabled {{
                background-color: #bdc3c7;
            }}
        '''
    
    def load_image(self):
        file_path, _ = QFileDialog.getOpenFileName(
            self,
            'Выберите изображение',
            '',
            'Изображения (*.png *.jpg *.jpeg *.bmp);;Все файлы (*.*)'
        )
        
        if file_path:
            try:
                test_img = cv2.imread(file_path)
                if test_img is None:
                    QMessageBox.critical(self, '❌ Ошибка', 'Не удалось загрузить изображение')
                    return
                
                self.current_image_path = file_path
                self.display_image(file_path, self.original_label)
                
                file_name = os.path.basename(file_path)
                file_size = os.path.getsize(file_path) / 1024
                file_size_str = f"{file_size:.1f} KB" if file_size < 1024 else f"{file_size/1024:.2f} MB"
                
                self.file_info.setText(f'📁 {file_name}\n📏 {file_size_str}')
                self.analyze_btn.setEnabled(True)
                self.status_bar.showMessage(f'✅ Загружено: {file_name}')
                
            except Exception as e:
                QMessageBox.critical(self, '❌ Ошибка', f'Ошибка: {str(e)}')
    
    def display_image(self, image_path, label_widget):
        pixmap = QPixmap(image_path)
        scaled_pixmap = pixmap.scaled(450, 350, Qt.KeepAspectRatio, Qt.SmoothTransformation)
        label_widget.setPixmap(scaled_pixmap)
    
    def analyze(self):
        if not self.current_image_path:
            return
        
        try:
            self.progress.show()
            self.progress.setValue(0)
            self.status_bar.showMessage('🔄 Анализ...')
            
            self.analyzer = LeafDiseaseAnalyzer(
                pixels_per_cm=62.0,
                min_leaf_area_cm=5.0
            )
            
            self.progress.setValue(30)
            QApplication.processEvents()
            
            self.results = self.analyzer.analyze_image(
                self.current_image_path,
                use_convex_hull=False
            )
            
            self.progress.setValue(80)
            QApplication.processEvents()
            
            # Отображаем результаты
            self.display_results()
            
            if 'visualization' in self.results:
                self.display_analysis_result(self.results['visualization'])
            
            # Добавляем в историю
            self.add_to_history()
            
            self.progress.setValue(100)
            self.status_bar.showMessage('✅ Анализ завершен')
            
        except Exception as e:
            QMessageBox.critical(self, '❌ Ошибка', f'Ошибка анализа:\n{str(e)}')
        finally:
            self.progress.hide()
    
    def display_analysis_result(self, vis_img):
        rgb_image = cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_image.shape
        bytes_per_line = ch * w
        qt_image = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qt_image)
        scaled_pixmap = pixmap.scaled(450, 350, Qt.KeepAspectRatio, Qt.SmoothTransformation)
        self.result_label.setPixmap(scaled_pixmap)
    
    def display_results(self):
        if not self.results:
            return
        
        text = []
        text.append("🍃 РЕЗУЛЬТАТЫ АНАЛИЗА")
        text.append("=" * 30)
        text.append(f"📊 Листьев: {self.results['total_leaves']}")
        text.append(f"📐 Общая площадь: {self.results['total_contour_area_cm']:.1f} см²")
        text.append(f"🍃 Здоровая: {self.results['total_healthy_area_cm']:.1f} см²")
        text.append(f"⚠️ Повреждения: {self.results['total_damage_area_cm']:.1f} см²")
        text.append(f"⚠️ Процент поражения: {self.results['overall_disease_percentage_damage']:.1f}%")
        
        self.results_text.setText('\n'.join(text))
    
    def add_to_history(self):
        """Добавление в историю"""
        filename = os.path.basename(self.current_image_path)
        date = datetime.now().strftime("%d.%m.%Y %H:%M")
        
        # Создаем миниатюру
        pixmap = QPixmap(self.current_image_path)
        thumbnail = pixmap.scaled(60, 60, Qt.KeepAspectRatio, Qt.SmoothTransformation)
        
        # Создаем элемент списка
        damage = self.results.get('overall_disease_percentage_damage', 0)
        leaves = self.results.get('total_leaves', 0)
        text = f"{filename}\n📅 {date}\n🍃 {leaves} лист. | ⚠️ {damage:.1f}%"
        
        item = QListWidgetItem()
        item.setText(text)
        item.setIcon(QIcon(QPixmap.fromImage(thumbnail.toImage())))
        item.setData(Qt.UserRole, {
            'filename': filename,
            'date': date,
            'image_path': self.current_image_path,
            'results': self.results
        })
        
        self.history_list.insertItem(0, item)
    
    def load_from_history(self, item):
        """Загрузка из истории"""
        data = item.data(Qt.UserRole)
        if data:
            self.current_image_path = data['image_path']
            self.results = data['results']
            
            # Отображаем
            self.display_image(self.current_image_path, self.original_label)
            self.display_analysis_result(self.results['visualization'])
            self.display_results()
            
            self.file_info.setText(f'📁 {data["filename"]}\n📅 {data["date"]}')
            self.analyze_btn.setEnabled(True)
            self.status_bar.showMessage(f'✅ Загружено из истории: {data["filename"]}')
    
    def clear_history(self):
        """Очистка истории"""
        reply = QMessageBox.question(self, 'Очистка истории', 
                                   'Очистить историю анализов?',
                                   QMessageBox.Yes | QMessageBox.No)
        if reply == QMessageBox.Yes:
            self.history_list.clear()


def main():
    app = QApplication(sys.argv)
    app.setStyle('Fusion')
    window = LeafDiseaseAnalyzerGUI()
    window.show()
    sys.exit(app.exec_())


if __name__ == '__main__':
    main()

SystemExit: 0

c:\Users\a.d.mikhaylov\.conda\envs\ann\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
